# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
path = os.path.join("../WeatherPy/output_data/cities.csv")
cities_data_frame = pd.read_csv(path)
cities_data_frame = cities_data_frame[["City", "Cloudiness", "Country" , "Date", "Humidity", "Lat", "Lng", "Max Temp","Wind Speed"]]
cities_data_frame

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,souillac,40,MU,1580954289,83,-20.52,57.52,299.15,4.60
1,punta arenas,100,CL,1580953994,70,-53.15,-70.92,282.15,2.44
2,bengkulu,89,ID,1580954174,61,-3.80,102.27,303.15,0.86
3,rio gallegos,20,AR,1580954292,57,-51.62,-69.22,283.15,3.10
4,bluff,42,NZ,1580954292,66,-46.60,168.33,287.59,7.15
...,...,...,...,...,...,...,...,...,...
568,vaitape,3,PF,1580954577,71,-16.52,-151.75,301.37,4.42
569,mujiayingzi,0,CN,1580954578,50,42.12,118.78,263.83,2.57
570,balkhash,100,KZ,1580954578,96,46.84,74.98,270.94,4.13
571,donskoye,100,RU,1580954579,60,45.46,41.97,278.15,5.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Setting the API key in gmaps configuration
gmaps.configure(api_key=g_key)
# Setting type of map
layout = {
    'height': '500px',
    'width': '100%'
}
fig = gmaps.figure(center=(0, 0), map_type='ROADMAP', zoom_level=2, layout=layout)

In [4]:
# Building dataframes needed
locations = cities_data_frame[["Lat", "Lng"]]
weights = cities_data_frame["Humidity"]

In [5]:
# Adding headmap layer
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights)
heatmap_layer.max_intensity = 400
heatmap_layer.point_radius = 20
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='500px', width='100%'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
hotel_df = cities_data_frame.loc[(cities_data_frame["Max Temp"] >= 294.261) & (cities_data_frame["Max Temp"] <= 299.817) & (cities_data_frame["Wind Speed"] < 10) & (cities_data_frame["Cloudiness"] == 0), :]
hotel_df = hotel_df.dropna()
hotel_df = hotel_df.reset_index(drop=all)
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,sao filipe,0,CV,1580954296,65,14.90,-24.50,295.50,6.82
1,chuy,0,UY,1580954302,82,-33.70,-53.46,296.44,6.46
2,port lincoln,0,AU,1580954307,60,-34.73,135.87,295.28,4.58
3,point pedro,0,LK,1580954353,66,9.82,80.23,299.65,4.58
4,mahbubnagar,0,IN,1580954359,60,16.73,77.98,295.26,3.19
5,bubaque,0,GW,1580954429,61,11.28,-15.83,299.64,1.69
6,jardim,0,BR,1580954421,80,-21.48,-56.14,297.23,1.90
7,angoche,0,MZ,1580954465,91,-16.23,39.91,296.21,1.73
8,santa maria,0,BR,1580954424,83,-29.68,-53.81,299.15,4.10
9,tuljapur,0,IN,1580954509,45,18.00,76.08,296.76,5.15


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
cities = hotel_df["City"]
hotels = list()
for city in cities:
    latitude = hotel_df.loc[hotel_df["City"] == city]["Lat"].reset_index(drop=all)[0]
    longitude = hotel_df.loc[hotel_df["City"] == city]["Lng"].reset_index(drop=all)[0]
    hotel_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?key={g_key}&location={latitude},{longitude}&radius=5000&type=hotel"
    hotel_response = requests.get(hotel_url).json()
    try:
        hotel = hotel_response["results"][1]["name"]
    except IndexError:
        hotel = None
        print("There's no hotels nearby")
    hotels.append(hotel)
hotels

['Tortuga B&B',
 'Nuevo Hotel Plaza',
 'Port Lincoln Hotel',
 'Mooththa Vinayakar Kovil',
 'Shankar Auto Agencies',
 'Casa Dora',
 'Hotel Estância',
 'BCI',
 'Hotel Morotin',
 'Hotel Son of Sun',
 'Argyle Hotel']

In [8]:
hotel_df["Hotel Name"] = hotels
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,sao filipe,0,CV,1580954296,65,14.90,-24.50,295.50,6.82,Tortuga B&B
1,chuy,0,UY,1580954302,82,-33.70,-53.46,296.44,6.46,Nuevo Hotel Plaza
2,port lincoln,0,AU,1580954307,60,-34.73,135.87,295.28,4.58,Port Lincoln Hotel
3,point pedro,0,LK,1580954353,66,9.82,80.23,299.65,4.58,Mooththa Vinayakar Kovil
4,mahbubnagar,0,IN,1580954359,60,16.73,77.98,295.26,3.19,Shankar Auto Agencies
5,bubaque,0,GW,1580954429,61,11.28,-15.83,299.64,1.69,Casa Dora
6,jardim,0,BR,1580954421,80,-21.48,-56.14,297.23,1.90,Hotel Estância
7,angoche,0,MZ,1580954465,91,-16.23,39.91,296.21,1.73,BCI
8,santa maria,0,BR,1580954424,83,-29.68,-53.81,299.15,4.10,Hotel Morotin
9,tuljapur,0,IN,1580954509,45,18.00,76.08,296.76,5.15,Hotel Son of Sun


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [10]:
# Add marker layer ontop of heat map
hotel_marker_layer = gmaps.marker_layer(locations=locations, info_box_content=hotel_info)
fig.add_layer(hotel_marker_layer)
# Display Map
fig

Figure(layout=FigureLayout(height='500px', width='100%'))